\begin{center} 
Chun-Yuan (Scott) Chiu 
\end{center}

\begin{center} 
chunyuac@andrew.cmu.edu 
\end{center} 

# 1. {-}

## (a) {-}

As the below realization of the error suggests, considering the mean squared prediction error, mean is a better estimator than median. 

In [103]:
from scipy.stats import expon
from numpy import abs
import numpy as np

mean = 1
median = np.log(2)
n = 1000

((expon.rvs(size=n) - mean)**2).mean(), ((expon.rvs(size=n) - median)**2).mean()

(1.0028184100775, 1.1652586921390726)

In contrast, considering the mean absolute prediction error, mean is a (slightly) better estimator than median. 

## (b) {-}

In [112]:
(abs(expon.rvs(size=n) - mean)).mean(), (abs(expon.rvs(size=n) - median)).mean()

(0.7651497131461258, 0.6929166469018724)